# 🛍️ Personalized Product Recommendation System
This notebook builds a recommendation engine using weighted user interactions (view vs. purchase).

In [ ]:
# 1. Import libraries
import pandas as pd
import numpy as np
import ast
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

In [ ]:
# 2. Load CSV files
product_df = pd.read_csv('product_recommendation_data.csv')
customer_df = pd.read_csv('customer_data_collection.csv')

In [ ]:
# 3. Clean and transform
product_df = product_df.drop(columns=['Unnamed: 13', 'Unnamed: 14'], errors='ignore')
customer_df = customer_df.drop(columns=['Unnamed: 10'], errors='ignore')
customer_df['Browsing_History'] = customer_df['Browsing_History'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])
customer_df['Purchase_History'] = customer_df['Purchase_History'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])

In [ ]:
# 4. Build interaction data with weights
interactions = []
for _, row in customer_df.iterrows():
    for product in row['Purchase_History']:
        interactions.append({'Customer_ID': row['Customer_ID'], 'Product_ID': product, 'interaction_type': 'purchase'})
    for product in row['Browsing_History']:
        interactions.append({'Customer_ID': row['Customer_ID'], 'Product_ID': product, 'interaction_type': 'view'})

interaction_df = pd.DataFrame(interactions)
interaction_df['interaction_value'] = interaction_df['interaction_type'].apply(lambda x: 2 if x == 'purchase' else 1)
user_item_df = interaction_df.groupby(['Customer_ID', 'Product_ID'])['interaction_value'].sum().reset_index()

In [ ]:
# 5. Encode IDs and build matrix
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()
user_item_df['user_idx'] = user_encoder.fit_transform(user_item_df['Customer_ID'])
user_item_df['item_idx'] = item_encoder.fit_transform(user_item_df['Product_ID'])
interaction_matrix = csr_matrix((user_item_df['interaction_value'], (user_item_df['user_idx'], user_item_df['item_idx'])))
item_similarity = cosine_similarity(interaction_matrix.T)

In [ ]:
# 6. Define recommendation function
def get_top_n_recommendations(user_id, interaction_matrix, similarity_matrix, n=5):
    user_idx = user_encoder.transform([user_id])[0]
    user_interactions = interaction_matrix[user_idx].toarray().flatten()
    scores = similarity_matrix.dot(user_interactions)
    known_items = interaction_matrix[user_idx].nonzero()[1]
    scores[known_items] = -np.inf
    top_items = np.argsort(scores)[-n:][::-1]
    return item_encoder.inverse_transform(top_items)

In [ ]:
# 7. Test with one user
example_user = user_item_df['Customer_ID'].iloc[0]
recommended_products = get_top_n_recommendations(example_user, interaction_matrix, item_similarity, n=5)
print(f"Recommended products for user {example_user}: {recommended_products}")

## 📌 Recommendation Model Summary

This notebook builds a personalized recommendation engine for an e-commerce platform using customer browsing and purchase data.

### 🎯 Objective
To improve the shopping experience and conversion rate by recommending the most relevant products to users based on their past behavior.

### ⚙️ Modeling Approach
- Weighted interactions (purchase = 2, view = 1)
- Sparse user-item matrix
- Item-item cosine similarity
- Recommend Top-N products that are most similar to previously interacted ones

### 🧪 Example Output
```python
Recommended products for user C1037: ['P042', 'P003', 'P099', 'P054', 'P082']
```